In [2]:
import pandas as pd
import requests

In [21]:
def get_reviews(appid, params):
        url_start = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url_start+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json() # return data extracted from the json response

In [63]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = { # https://partner.steamgames.com/doc/store/getreviews
            'json' : 1,
            'filter' : 'all', # sort by: recent, updated, all (helpfullness)
            'language' : 'english', # https://partner.steamgames.com/doc/store/localization
            'day_range' : 9223372036854775807, # shows reveiws from all time
            'review_type' : 'all', # all, positive, negative
            'purchase_type' : 'all', # all, non_steam_purchase, steam
        }
    while n > 0:
        params['cursor'] = cursor.encode() # for pagination
        params['num_per_page'] = min(100, n) # 100 is the max possible reviews in one requests
        n -= 100
        
        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
        
        if len(response['reviews']) < 100: break
    
    return reviews

In [68]:
reviews[0]

{'recommendationid': '81938368',
 'author': {'steamid': '76561198036857181',
  'num_games_owned': 1102,
  'num_reviews': 27,
  'playtime_forever': 2581,
  'playtime_last_two_weeks': 495,
  'playtime_at_review': 207,
  'last_played': 1613719739},
 'language': 'english',
 'review': '[h1]DOWNLOAD THE NVIDIA GAME READY DRIVERS FOR THE LOVE OF GOD[/h1]\nI crashed like twice but didn\'t have a problem after I got the new game ready drivers. I\'m literally making this review as a reminder. Had a blast so far but jesus guys, update your nvidia drivers.\n\nULTRA EDIT:\n[h1]A Flawed but Fun Experience[/h1]\n\nI might as well actually review this game in its current state considering the attention this got, while I have not finished it I feel I have experienced enough of the core mechanics to formulate an accurate opinion at least on the gameplay. \n\nIt is a Witcher 3 reskin with a cyberpunk setting, considering that Witcher 3 was amazing (and I still have yet to get through it entirely) it does

In [65]:
reviews = []
appids = ['1091500', '1545280'] # Cyberpunk 2077, good mix of positive and negative, just for samples
for appid in appids:
    reviews += get_n_reviews(appid, 1000)
df = pd.DataFrame(reviews)[['review', 'voted_up']]
df

,review,voted_up
0,[h1]DOWNLOAD THE NVIDIA GAME READY DRIVERS FOR...,True
1,"Although its buggy and feels poorly optimized,...",True
2,if you have an RTX card turn on DLSS. helps ou...,True
3,I love this game but games needs to be optimiz...,True
4,[h1]Just remember:[/h1]\n[olist]\n[*][b]Angel ...,True
...,...,...
998,By far the number 1 game I have played in a lo...,True
999,"yeah had s3x with a dude loved it,.",True
1000,"Quite a difficult arcade game, not for reactio...",True
1001,"A mixture of puzzle, runner and physical sandb...",True


In [ ]:
df.to_csv('data\reviews.csv', index=False)